In [1]:
import tensorflow as tf

2024-03-22 15:55:46.455384: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 15:55:46.924121: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 15:55:48.103477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.is_built_with_cuda():
    print("TensorFlow was built with CUDA")
else:
    print("TensorFlow was not built with CUDA")

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

2024-03-22 16:31:23.534688: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 16:31:24.105975: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 16:31:26.537510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0
TensorFlow was built with CUDA
Please install GPU version of TF


2024-03-22 16:31:29.045329: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 16:31:29.270352: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-03-22 16:31:29.273684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 16:31:29.273710: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the 

In [4]:
import glob
import os

In [7]:
m="/home/wailmouro/code/benoitfrisque/signlens/training_outputs/model 20240325-120044"
keras_files = glob.glob(os.path.join(m,'model', "*.keras"))
keras_files

[]

In [2]:
!echo $EPOCHS

500


In [1]:
%load_ext autoreload
%autoreload 2

from signlens.preprocessing.data import load_data_subset_csv, unique_train_test_split,filter_out_landmarks,load_relevant_data_subset


In [2]:
train = load_data_subset_csv(balanced=True, random_state=None)

Loading data subset from train_train.csv
    ℹ️ Filtered sequences with missing frames. Size reduced from 77228 to 77228 (100.00%)
    ℹ️ Filtered on n_frames = 100. Size reduced from 77228 to 68935 (89.26%)
    ℹ️ Filtered on n_classes = 250. Size reduced from 68935 to 68935 (100.00%)
    ℹ️ Balanced data, with average of 27.6 elements per class. Size reduced from 68935 to 6893 (10.00%)
✅ Loaded 6893 rows (8.93% of the original 77228 rows) from the dataset.


In [12]:
from signlens.params import *
from signlens.preprocessing.data import load_data_subset_csv, unique_train_test_split
from signlens.preprocessing.preprocess import preprocess_and_pad_sequences_from_pq_list, encode_labels
from signlens.model.model import initialize_model, compile_model, train_model, evaluate_model
from utils.model_utils import save_results, save_model, load_model, create_model_folder
from sklearn.model_selection import train_test_split

In [13]:
train

,path,participant_id,sequence_id,sign,n_frames,n_frames2,file_path
0,train_landmark_files_noface/29302/192708374.pa...,29302,192708374,closet,34,34,/home/wailmouro/code/benoitfrisque/signlens/ra...
1,train_landmark_files_noface/49445/2771927098.p...,49445,2771927098,weus,53,53,/home/wailmouro/code/benoitfrisque/signlens/ra...
2,train_landmark_files_noface/53618/3591372939.p...,53618,3591372939,hungry,8,8,/home/wailmouro/code/benoitfrisque/signlens/ra...
3,train_landmark_files_noface/16069/1501987033.p...,16069,1501987033,helicopter,27,27,/home/wailmouro/code/benoitfrisque/signlens/ra...
4,train_landmark_files_noface/53618/2217436702.p...,53618,2217436702,cry,96,96,/home/wailmouro/code/benoitfrisque/signlens/ra...
...,...,...,...,...,...,...,...
6888,train_landmark_files_noface/53618/383752124.pa...,53618,383752124,underwear,14,14,/home/wailmouro/code/benoitfrisque/signlens/ra...
6889,train_landmark_files_noface/32319/286899901.pa...,32319,286899901,noisy,54,54,/home/wailmouro/code/benoitfrisque/signlens/ra...
6890,train_landmark_files_noface/37055/2293931034.p...,37055,2293931034,flower,14,14,/home/wailmouro/code/benoitfrisque/signlens/ra...
6891,train_landmark_files_noface/29302/2782983417.p...,29302,2782983417,nuts,37,37,/home/wailmouro/code/benoitfrisque/signlens/ra...


In [15]:
X_files = train.file_path
y = encode_labels(train.sign)
X_train_files, X_val_files, y_train, y_val = train_test_split(X_files, y, test_size=0.2, stratify=y, random_state=None)

In [17]:
load_relevant_data_subset(X_train_files[0])

NameError: name 'load_relevant_data_subset' is not defined

In [8]:
filter_out_landmarks(train.file_path[0],"pose")+


pose
hey
hey
hey
hey


,frame,row_id,type,landmark_index,x,y,z
489,6,6-pose-0,pose,0,0.509889,0.399118,-1.545641
490,6,6-pose-1,pose,1,0.559853,0.356564,-1.509517
491,6,6-pose-2,pose,2,0.585243,0.359251,-1.509425
492,6,6-pose-3,pose,3,0.607828,0.362500,-1.509716
493,6,6-pose-4,pose,4,0.488029,0.353529,-1.491490
...,...,...,...,...,...,...,...
18426,39,39-pose-18,pose,18,0.240427,0.719300,-2.071481
18427,39,39-pose-19,pose,19,0.756227,0.576663,-1.208361
18428,39,39-pose-20,pose,20,0.269345,0.684207,-2.012549
18429,39,39-pose-21,pose,21,0.740706,0.605528,-1.176791
